In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential


gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Prevent memory overflow

        if len(gpus) > 1:
            strategy = tf.distribute.MirroredStrategy()  # Use multiple GPUs (T4 x2 case)
            print(f"Using Multi-GPU Training with {len(gpus)} GPUs")
        else:
            strategy = tf.distribute.OneDeviceStrategy(device="/GPU:0")  # Single GPU (P100 case)
            print("Using Single-GPU Training")

    except RuntimeError as e:
        print(e)
else:
    strategy = tf.distribute.OneDeviceStrategy(device="/CPU:0")  # Fallback to CPU if no GPU
    print("Using CPU (No GPU detected)")

# Data paths
dataset_path = "/kaggle/input/train1/Train1"  # Update with your dataset path

# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)


with strategy.scope():
    
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

    
    for layer in base_model.layers:
        layer.trainable = False

    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x) 

    model = Model(inputs=base_model.input, outputs=x)

    
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


model.fit(train_generator, validation_data=val_generator, epochs=10)


model.save("resnet_accident_classifier.h5")
print("Model training complete and saved!")


Using Multi-GPU Training with 2 GPUs
Found 6401 images belonging to 2 classes.
Found 1599 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 102s 436ms/step - accuracy: 0.6837 - loss: nan - val_accuracy: 0.7675 - val_loss: 0.4829
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 60s 292ms/step - accuracy: 0.8904 - loss: nan - val_accuracy: 0.7088 - val_loss: 0.5333
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 60s 288ms/step - accuracy: 0.9264 - loss: nan - val_accuracy: 0.7450 - val_loss: 0.4873
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 60s 290ms/step - accuracy: 0.9291 - loss: nan - val_accuracy: 0.5900 - val_loss: 0.8843
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 60s 290ms/step - accuracy: 0.9316 - loss: nan - val_accuracy: 0.6050 - val_loss: 0.8908
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 60s 289ms/step - accuracy: 0.9437 - loss: nan - val_accuracy: 0.6587 - val_loss: 0.7838
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 59s 287ms/step - accuracy: 0.9474 - loss: nan - val_accuracy: 0.6112 - val_loss: 1.0520
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 60s 292ms/step - accuracy: 0.9548 - loss: nan - val_accuracy: 0

In [10]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


x = GlobalAveragePooling2D()(base_model.output)  
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  


model = Model(inputs=base_model.input, outputs=x)


model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])




In [11]:
model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 84s 350ms/step - accuracy: 0.7485 - loss: 0.5273 - val_accuracy: 0.5810 - val_loss: 0.8596
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 56s 271ms/step - accuracy: 0.9077 - loss: 0.2321 - val_accuracy: 0.7217 - val_loss: 0.5589
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 56s 272ms/step - accuracy: 0.9296 - loss: 0.1749 - val_accuracy: 0.6560 - val_loss: 0.8350
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 56s 271ms/step - accuracy: 0.9479 - loss: 0.1390 - val_accuracy: 0.7317 - val_loss: 0.6479
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 57s 274ms/step - accuracy: 0.9220 - loss: 0.1847 - val_accuracy: 0.5553 - val_loss: 1.5253
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 56s 267ms/step - accuracy: 0.9490 - loss: 0.1295 - val_accuracy: 0.5985 - val_loss: 1.2708
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 56s 269ms/step - accuracy: 0.9586 - loss: 0.0995 - val_accuracy: 0.6054 - val_loss: 1.1868
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 54s 261ms/step - accuracy: 0.9571 - loss: 0

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import AdamW
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers[-50:]:
    layer.trainable = True


x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x) 
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/train1/Train1",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    "/kaggle/input/train1/Train1",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

class_labels = train_generator.classes
class_weights = compute_class_weight("balanced", classes=np.unique(class_labels), y=class_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


model.compile(
    optimizer=AdamW(learning_rate=1e-4, weight_decay=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, verbose=1
)


model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weight_dict,  
    callbacks=[lr_scheduler]
)


model.save("resnet_accident_classifier_finetunedhh.h5")
print("✅ Model Training Complete!")


Found 6401 images belonging to 2 classes.
Found 1599 images belonging to 2 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


201/201 ━━━━━━━━━━━━━━━━━━━━ 216s 765ms/step - accuracy: 0.9160 - loss: 0.1581 - val_accuracy: 0.4115 - val_loss: 2.0153 - learning_rate: 1.0000e-04
Epoch 2/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 126s 612ms/step - accuracy: 0.9940 - loss: 0.0193 - val_accuracy: 0.4115 - val_loss: 1.5609 - learning_rate: 1.0000e-04
Epoch 3/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 128s 618ms/step - accuracy: 0.9897 - loss: 0.0288 - val_accuracy: 0.4115 - val_loss: 4.0186 - learning_rate: 1.0000e-04
Epoch 4/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 127s 616ms/step - accuracy: 0.9990 - loss: 0.0041 - val_accuracy: 0.6904 - val_loss: 1.9063 - learning_rate: 1.0000e-04
Epoch 5/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - accuracy: 0.9937 - loss: 0.0258
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
201/201 ━━━━━━━━━━━━━━━━━━━━ 127s 615ms/step - accuracy: 0.9937 - loss: 0.0258 - val_accuracy: 0.7824 - val_loss: 1.6397 - learning_rate: 1.0000e-04
Epoch 6/20
201/201 ━━━━━━━━━━━━━━━━━━━━ 128s 619ms/step - 

In [13]:
model.save("train1.h5")